### **Load dataset and clean it**

In [15]:
import pandas as pd
import numpy as np

# Define columns that should be numeric
numeric_cols = [
    'BATTING_Tests_Mat', 'BATTING_Tests_Runs', 'BATTING_Tests_Ave',
    'BOWLING_Tests_Mat', 'BOWLING_Tests_Wkts', 'BOWLING_Tests_Ave',
    'BATTING_ODIs_Mat', 'BATTING_ODIs_Runs', 'BATTING_ODIs_Ave',
    'BOWLING_ODIs_Mat', 'BOWLING_ODIs_Wkts', 'BOWLING_ODIs_Ave',
    'BATTING_T20Is_Mat', 'BATTING_T20Is_Runs', 'BATTING_T20Is_SR',
    'BOWLING_T20Is_Mat', 'BOWLING_T20Is_Wkts', 'BOWLING_T20Is_Econ'
]

# Load data with proper type conversion
def convert_to_float(x):
    try:
        # Handle percentages, commas, etc.
        x = str(x).replace(',', '').replace('%', '')
        return float(x) if x.replace('.', '').isdigit() else np.nan
    except:
        return np.nan

df = pd.read_csv('/workspaces/Cricket_website_ML_project/data/cricket_data.csv')

# Convert numeric columns
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].apply(convert_to_float)
    else:
        df[col] = np.nan  # Create column if missing

def restructure_data(df):
    # Initialize new dataframe with basic info
    players = df[['ID', 'NAME', 'COUNTRY', 'Full name', 'Born', 'Died', 
                 'Current age', 'Major teams', 'Playing role', 'Batting style', 
                 'Bowling style']].copy()
    
    # Add gender identification (we'll implement this next)
    players['gender'] = None
    
    # Define cricket formats
    formats = ['Tests', 'ODIs', 'T20Is', 'First-class', 'List A', 'T20s']
    
    # Create nested batting and bowling stats
    for fmt in formats:
        # Batting stats
        bat_cols = [f'BATTING_{fmt}_{stat}' for stat in 
                   ['Mat', 'Inns', 'NO', 'Runs', 'HS', 'Ave', 'BF', 'SR',
                    '100', '50', '4s', '6s', 'Ct', 'St']]
        
        # Bowling stats
        bowl_cols = [f'BOWLING_{fmt}_{stat}' for stat in 
                    ['Mat', 'Inns', 'Balls', 'Runs', 'Wkts', 'BBI', 'BBM',
                     'Ave', 'Econ', 'SR', '4w', '5w', '10']]
        
        # Create nested dictionaries
        players[f'{fmt.lower()}_batting'] = df[bat_cols].to_dict('records')
        players[f'{fmt.lower()}_bowling'] = df[bowl_cols].to_dict('records')
    
    return players

restructured_df = restructure_data(df)

/tmp/ipykernel_38286/91566234.py:23: DtypeWarning: Columns (9,16,20,21,22,24,25,26,27,28,29,30,31,32,45,46,59,60,73,74,87,88,90,91,92,95,96,97,98,99,100,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/workspaces/Cricket_website_ML_project/data/cricket_data.csv')


### **Gender Indentification**

In [16]:
def identify_gender(df):
    # Initialize gender column
    df['gender'] = 'male'  # Default assumption
    
    # Rule 1: Check if "women" appears in team names (more comprehensive)
    women_teams = df['Major teams'].str.contains(
        r'women|Women|Ladies|ladies|Wmn|wmn', 
        case=False, na=False
    )
    
    # Rule 2: Check for female pronouns in description (if column exists)
    female_pronouns = pd.Series(False, index=df.index)
    if 'DESCRIPTION' in df.columns:
        female_pronouns = df['DESCRIPTION'].str.contains(
            r'\b(she|her|hers)\b', 
            case=False, na=False
        )
    
    # Rule 3: Known female name patterns (without regex groups)
    female_name_patterns = [
        'Sharmin', 'Mithali', 'Smriti', 'Meg', 'Ellyse', 'Sophie',
        'Harmanpreet', 'Jemimah', 'Deepti', 'Poonam', 'Rajeshwari',
        'Amy', 'Beth', 'Charlotte', 'Danni', 'Heather'
    ]
    female_names = df['NAME'].str.contains(
        '|'.join(female_name_patterns), 
        case=False, na=False
    )
    
    # Apply rules - any positive indicates female player
    df.loc[women_teams | female_pronouns | female_names, 'gender'] = 'female'
    
    return df

### **Cricket Features**

In [17]:
def add_cricket_metrics(df):
    # Calculate overall batting/bowling metrics
    formats = ['tests', 'odis', 't20is']
    
    for fmt in formats:
        # Batting strike rate impact - handle missing/nested data properly
        df[f'{fmt}_batting_impact'] = df[f'{fmt}_batting'].apply(
            lambda x: x.get('BATTING_'+fmt.upper()+'_SR', 0) * x.get('BATTING_'+fmt.upper()+'_Ave', 0) 
            if isinstance(x, dict) and pd.notna(x.get('BATTING_'+fmt.upper()+'_SR')) 
            and pd.notna(x.get('BATTING_'+fmt.upper()+'_Ave')) else 0)
        
        # Bowling economy impact
        df[f'{fmt}_bowling_impact'] = df[f'{fmt}_bowling'].apply(
            lambda x: (100 - x.get('BOWLING_'+fmt.upper()+'_Econ', 100)) * x.get('BOWLING_'+fmt.upper()+'_Wkts', 0) 
            if isinstance(x, dict) and pd.notna(x.get('BOWLING_'+fmt.upper()+'_Econ')) 
            and pd.notna(x.get('BOWLING_'+fmt.upper()+'_Wkts')) else 0)
    
    # Era classification with proper birth year extraction
    df['birth_year'] = df['Born'].str.extract(r'(\d{4})')[0].astype(float)
    df['era'] = pd.cut(
        df['birth_year'],
        bins=[1800, 1920, 1970, 1990, 2010, 2025],
        labels=['Pre-WWII', 'Golden Age', 'Modern', 'Contemporary', 'Current'],
        right=False
    )
    
    return df

### **Identifying Allrounders and removing duplicates**

In [18]:
def identify_allrounders(df):
    def safe_compare(val, threshold):
        try:
            return float(val) >= threshold
        except:
            return False
    
    def test_allrounder(row):
        return (safe_compare(row['BATTING_Tests_Runs'], 1000) and 
               safe_compare(row['BATTING_Tests_Ave'], 25) and
               safe_compare(row['BOWLING_Tests_Wkts'], 100))
    
    def odi_allrounder(row):
        return (safe_compare(row['BATTING_ODIs_Runs'], 1000) and 
               safe_compare(row['BATTING_ODIs_Ave'], 25) and
               safe_compare(row['BOWLING_ODIs_Wkts'], 80))
    
    def t20_allrounder(row):
        return (safe_compare(row['BATTING_T20Is_Runs'], 250) and 
               safe_compare(row['BOWLING_T20Is_Wkts'], 25))
    
    df['is_test_ar'] = df.apply(test_allrounder, axis=1)
    df['is_odi_ar'] = df.apply(odi_allrounder, axis=1)
    df['is_t20_ar'] = df.apply(t20_allrounder, axis=1)
    
    df['is_allrounder'] = df['is_test_ar'] | df['is_odi_ar'] | df['is_t20_ar']
    return df

df = identify_allrounders(df)

In [19]:
def deduplicate_players(df):
    # First clean the NAME and COUNTRY columns
    df['NAME'] = df['NAME'].str.strip()
    df['COUNTRY'] = df['COUNTRY'].str.strip()
    
    # Group by cleaned names and country
    grouped = df.groupby(['NAME', 'COUNTRY'], as_index=False)
    
    # Define aggregation - for each column type
    agg_dict = {}
    for col in df.columns:
        if col in ['ID', 'NAME', 'COUNTRY']:
            continue  # These are our grouping columns
        elif pd.api.types.is_numeric_dtype(df[col]):
            agg_dict[col] = 'max'
        else:
            agg_dict[col] = 'first'
    
    # Perform the aggregation
    deduped = grouped.agg(agg_dict)
    return deduped

### Sample Check

In [20]:
# Check data types
print("\nData types:")
print(df[numeric_cols].dtypes)

# Check sample values
print("\nSample values:")
print(df[numeric_cols].head())

# Check known all-rounders
known_ar = ['Jacques Kallis', 'Imran Khan', 'Kapil Dev', 'Shakib Al Hasan']
for name in known_ar:
    player = df[df['NAME'].str.contains(name, case=False, na=False)]
    if not player.empty:
        print(f"\n{name}:")
        print(f"Test Batting - Mat: {player['BATTING_Tests_Mat'].values[0]}, Avg: {player['BATTING_Tests_Ave'].values[0]}")
        print(f"Test Bowling - Mat: {player['BOWLING_Tests_Mat'].values[0]}, Wkts: {player['BOWLING_Tests_Wkts'].values[0]}")
        print(f"All-rounder status: {player['is_allrounder'].values[0]}")
    else:
        print(f"\n{name} not found in dataset")


Data types:
BATTING_Tests_Mat     float64
BATTING_Tests_Runs    float64
BATTING_Tests_Ave     float64
BOWLING_Tests_Mat     float64
BOWLING_Tests_Wkts    float64
BOWLING_Tests_Ave     float64
BATTING_ODIs_Mat      float64
BATTING_ODIs_Runs     float64
BATTING_ODIs_Ave      float64
BOWLING_ODIs_Mat      float64
BOWLING_ODIs_Wkts     float64
BOWLING_ODIs_Ave      float64
BATTING_T20Is_Mat     float64
BATTING_T20Is_Runs    float64
BATTING_T20Is_SR      float64
BOWLING_T20Is_Mat     float64
BOWLING_T20Is_Wkts    float64
BOWLING_T20Is_Econ    float64
dtype: object

Sample values:
   BATTING_Tests_Mat  BATTING_Tests_Runs  BATTING_Tests_Ave  \
0                NaN                 NaN                NaN   
1                NaN                 NaN                NaN   
2                NaN                 NaN                NaN   
3                NaN                 NaN                NaN   
4                NaN                 NaN                NaN   

   BOWLING_Tests_Mat  BOWLING_Tests_Wk

In [21]:
# Fill missing values appropriately
for col in numeric_cols:
    if 'Mat' in col or 'Wkts' in col or 'Runs' in col:
        df[col].fillna(0, inplace=True)
    elif 'Ave' in col or 'SR' in col or 'Econ' in col:
        df[col].fillna(df[col].median(), inplace=True)

# Verify no nulls remain
print("\nMissing values after cleaning:")
print(df[numeric_cols].isnull().sum())


Missing values after cleaning:
BATTING_Tests_Mat     0
BATTING_Tests_Runs    0
BATTING_Tests_Ave     0
BOWLING_Tests_Mat     0
BOWLING_Tests_Wkts    0
BOWLING_Tests_Ave     0
BATTING_ODIs_Mat      0
BATTING_ODIs_Runs     0
BATTING_ODIs_Ave      0
BOWLING_ODIs_Mat      0
BOWLING_ODIs_Wkts     0
BOWLING_ODIs_Ave      0
BATTING_T20Is_Mat     0
BATTING_T20Is_Runs    0
BATTING_T20Is_SR      0
BOWLING_T20Is_Mat     0
BOWLING_T20Is_Wkts    0
BOWLING_T20Is_Econ    0
dtype: int64


In [22]:
# Get confirmed all-rounders
allrounders = df[df['is_allrounder']][['NAME', 'COUNTRY', 'is_test_ar', 'is_odi_ar', 'is_t20_ar']]
print(f"\nFound {len(allrounders)} all-rounders:")
print(allrounders.head(20))

# Save results
allrounders.to_csv('/workspaces/Cricket_website_ML_project/data/allrounders_list.csv', index=False)


Found 116 all-rounders:
                      NAME               COUNTRY  is_test_ar  is_odi_ar  \
3043           Laura Marsh               England       False      False   
4644             Moeen Ali               England        True      False   
5721    Nattaya Boochatham              Thailand       False      False   
8364         Mohammad Nabi           Afghanistan       False       True   
8817    Samiullah Shinwari           Afghanistan       False      False   
12150         Isobel Joyce               Ireland       False      False   
12894     Mashrafe Mortaza            Bangladesh       False      False   
12928          Mahmudullah            Bangladesh       False      False   
13122         Rumana Ahmed            Bangladesh       False      False   
13378      Shakib Al Hasan            Bangladesh        True       True   
13517         Salma Khatun            Bangladesh       False      False   
14057         Steve Tikolo                 Kenya       False       True   


### **Save the cleaned df**

In [23]:
import json
# 1. Save Main DataFrame
# ----------------------
# Flatten nested stats for CSV
flattened_df = pd.json_normalize(
    df.to_dict('records'),
    sep='_'
)

# Save to CSV with proper data types
flattened_df.to_csv('/workspaces/Cricket_website_ML_project/data/flattened_cricket_data.csv', index=False)

# 2. Save Structured JSON (for web app)
# ------------------------------------
# Keep nested structure
with open('/workspaces/Cricket_website_ML_project/data/structured_cricket_data.json', 'w') as f:
    json.dump(df.to_dict('records'), f, indent=2)

# 3. Save Specialized DataFrames
# -----------------------------
# Batting-focused data
batting_df = df[[
    'ID', 'NAME', 'COUNTRY', 'Playing role',
    'BATTING_Tests_Mat', 'BATTING_Tests_Runs', 'BATTING_Tests_Ave',
    'BATTING_ODIs_Mat', 'BATTING_ODIs_Runs', 'BATTING_ODIs_SR',
    'BATTING_T20Is_Mat', 'BATTING_T20Is_Runs', 'BATTING_T20Is_SR'
]].copy()
batting_df.to_csv('/workspaces/Cricket_website_ML_project/data/batting_stats.csv', index=False)

# Bowling-focused data
bowling_df = df[[
    'ID', 'NAME', 'COUNTRY', 'Playing role',
    'BOWLING_Tests_Mat', 'BOWLING_Tests_Wkts', 'BOWLING_Tests_Ave',
    'BOWLING_ODIs_Mat', 'BOWLING_ODIs_Wkts', 'BOWLING_ODIs_Econ',
    'BOWLING_T20Is_Mat', 'BOWLING_T20Is_Wkts', 'BOWLING_T20Is_Econ'
]].copy()
bowling_df.to_csv('/workspaces/Cricket_website_ML_project/data/bowling_stats.csv', index=False)

# 4. Save Format-Specific Data
# ---------------------------
# Test specialists
test_players = df[
    (df['BATTING_Tests_Mat'] > 10) | 
    (df['BOWLING_Tests_Mat'] > 10)
]
test_players.to_csv('/workspaces/Cricket_website_ML_project/data/test_specialists.csv', index=False)

# ODI specialists
odi_players = df[
    (df['BATTING_ODIs_Mat'] > 20) | 
    (df['BOWLING_ODIs_Mat'] > 20)
]
odi_players.to_csv('/workspaces/Cricket_website_ML_project/data/odi_specialists.csv', index=False)

# T20 specialists
t20_players = df[
    (df['BATTING_T20Is_Mat'] > 10) | 
    (df['BOWLING_T20Is_Mat'] > 10)
]
t20_players.to_csv('/workspaces/Cricket_website_ML_project/data/t20_specialists.csv', index=False)

# 5. Save Metadata
# ----------------
import yaml

metadata = {
    'last_updated': pd.Timestamp.now().isoformat(),
    'records_count': len(df),
    'columns': list(df.columns),
    'allrounder_count': len(allrounders)
}

with open('/workspaces/Cricket_website_ML_project/data/metadata.yaml', 'w') as f:
    yaml.dump(metadata, f)